### Segmenting and analysing Chicago Schools

Although growing national attention has focused on high rates of required participation in college admission, I reviewed, Chicago, included college enrollments rates in its rating systems. Admission and enrollments rates are generally collected state by state through statewide college reporting databases from each schools.
Although there is increased nationwide focus on college and career readiness, state and national data systems cannot always provide the necessary data to evaluate schools on readiness. Many states are prioritizing efforts to link K–6 with postsecondary and state data systems in order to follow students from elementry school through highschool and college or employment. In addition to state data sources, college data may be accessed from testing agencies such as ACT and the
College Board, and from the National Student Clearinghouse, which calculates college attendance rates. In this section we focused on data from city of Chicago, to revise which community is the top and with communities are better for great schools.

In [108]:
#Import libraries

!pip install geopy
!pip install folium
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualization
import matplotlib.pyplot
import seaborn as sns
# Too see full dataframe...
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
# First We have to locate the file path and changed accordingly

Libraries imported.


## Load data and prepare it for Analysing and Clustering

#### Dataset belongs to chicago's school and has about 15 columns. I like to prepare it for clustering as best schools in chicago's communities

In [56]:
Schools=pd.read_csv('D:\DataScience\DataFrames\SCHOOLS-chicago.csv')

In [57]:
Schools.head()

,School ID,NAME_OF_SCHOOL,"Elementary, Middle, or High School",ZIP Code,CPS Performance Policy Level,SAFETY_SCORE,Instruction Score,Leaders Score,Teachers Score,Rate of Misconducts (per 100 students),COLLEGE_ENROLLMENT,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
0,610038,Abraham Lincoln Elementary School,ES,60614,Level 1,99.0,66.0,65,70,2.0,813,41.924497,-87.644522,7,LINCOLN PARK
1,610281,Adam Clayton Powell Paideia Community Academy ...,ES,60649,Level 1,54.0,84.0,63,76,15.7,521,41.760324,-87.556736,43,SOUTH SHORE
2,610185,Adlai E Stevenson Elementary School,ES,60652,Level 2,61.0,36.0,NDA,NDA,2.3,1324,41.747111,-87.731702,70,ASHBURN
3,609993,Agustin Lara Elementary Academy,ES,60609,Level 1,56.0,37.0,65,48,10.4,556,41.809757,-87.672145,61,NEW CITY
4,610513,Air Force Academy High School,HS,60609,Not Enough Data,49.0,55.0,45,54,15.6,302,41.828146,-87.632794,34,ARMOUR SQUARE


In [58]:
Schools.columns=Schools.columns.str.replace(' ','_')##### replace blanks with _ to reach column name properly

In [59]:
Schools.shape

(566, 15)

In [47]:
Schools

,School_ID,NAME_OF_SCHOOL,"Elementary,_Middle,_or_High_School",ZIP_Code,CPS_Performance_Policy_Level,SAFETY_SCORE,Instruction_Score,Leaders_Score_,Teachers_Score,Rate_of_Misconducts_(per_100_students)_,COLLEGE_ENROLLMENT,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
0,610038,Abraham Lincoln Elementary School,ES,60614,Level 1,99.0,66.0,65,70,2.0,813,41.924497,-87.644522,7,LINCOLN PARK
1,610281,Adam Clayton Powell Paideia Community Academy ...,ES,60649,Level 1,54.0,84.0,63,76,15.7,521,41.760324,-87.556736,43,SOUTH SHORE
2,610185,Adlai E Stevenson Elementary School,ES,60652,Level 2,61.0,36.0,NDA,NDA,2.3,1324,41.747111,-87.731702,70,ASHBURN
3,609993,Agustin Lara Elementary Academy,ES,60609,Level 1,56.0,37.0,65,48,10.4,556,41.809757,-87.672145,61,NEW CITY
4,610513,Air Force Academy High School,HS,60609,Not Enough Data,49.0,55.0,45,54,15.6,302,41.828146,-87.632794,34,ARMOUR SQUARE
5,610212,Albany Park Multicultural Academy,MS,60625,Level 1,66.0,71.0,43,50,2.3,266,41.971143,-87.709627,14,ALBANY PARK
6,609720,Albert G Lane Technical High School,HS,60618,Level 1,88.0,52.0,NDA,NDA,2.1,4368,41.946617,-87.691056,5,NORTH CENTER
7,610342,Albert R Sabin Elementary Magnet School,ES,60622,Level 3,67.0,18.0,NDA,NDA,28.1,620,41.906843,-87.683043,24,WEST TOWN
8,610524,Alcott High School for the Humanities,HS,60618,Not Enough Data,70.0,51.0,NDA,NDA,7.1,232,41.935761,-87.680524,5,NORTH CENTER
9,610209,Alessandro Volta Elementary School,ES,60625,Level 2,43.0,37.0,62,56,22.5,1023,41.971606,-87.724641,14,ALBANY PARK


In [25]:
#Schools=Schools[Schools.Leaders_Score_ !='NDA']

#### Make a copy of dataset to manage clustering on it

In [60]:
schools=Schools
schools.sort_values('COLLEGE_ENROLLMENT', ascending=False)# sort school based on college enrollment
schools.head()

,School_ID,NAME_OF_SCHOOL,"Elementary,_Middle,_or_High_School",ZIP_Code,CPS_Performance_Policy_Level,SAFETY_SCORE,Instruction_Score,Leaders_Score_,Teachers_Score,Rate_of_Misconducts_(per_100_students)_,COLLEGE_ENROLLMENT,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME
0,610038,Abraham Lincoln Elementary School,ES,60614,Level 1,99.0,66.0,65,70,2.0,813,41.924497,-87.644522,7,LINCOLN PARK
1,610281,Adam Clayton Powell Paideia Community Academy ...,ES,60649,Level 1,54.0,84.0,63,76,15.7,521,41.760324,-87.556736,43,SOUTH SHORE
2,610185,Adlai E Stevenson Elementary School,ES,60652,Level 2,61.0,36.0,NDA,NDA,2.3,1324,41.747111,-87.731702,70,ASHBURN
3,609993,Agustin Lara Elementary Academy,ES,60609,Level 1,56.0,37.0,65,48,10.4,556,41.809757,-87.672145,61,NEW CITY
4,610513,Air Force Academy High School,HS,60609,Not Enough Data,49.0,55.0,45,54,15.6,302,41.828146,-87.632794,34,ARMOUR SQUARE


In [7]:
schools.COLLEGE_ENROLLMENT.min()

21

In [66]:
########## these two cells contain codes to replace NDA with average value of them.###########
for i,row in  schools.iterrows():
    if(schools.loc[i,'Leaders_Score_']=='NDA'):
        schools.loc[i,'Leaders_Score_']=schools.['Leaders_Score_'].mean()


In [64]:
for i,row in  schools.iterrows():
    if(schools.loc[i,'Teachers_Score']=='NDA'):
        schools.loc[i,'Teachers_Score']=schools.['Leaders_Score_'].mean()

#### Get the latitude and longitude of Chicago city to map it and add schools

In [7]:
address = 'Chicago, IL'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago are 41.8755616, -87.6244212.


#### Now let's Visualize chicago with schools location

In [70]:
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(schools['Latitude'], schools['Longitude'], schools['COMMUNITY_AREA_NAME']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

In [24]:
schools=schools.rename(columns={'Rate_of_Misconducts_(per_100_students)_':'Rate_of_Misconducts(per_100_students)'}, inplace=True)

## 2. Exploring schools based on College enrollment 

For categories schools from point of College-enrollment, I grouped the schools to find better schools in term of performance.
As you can see above the schools which have high college enrollment are those have somehow minimum Rate of Misconducts and beside that, those schools have about more than 50% safety score.

I categorize scools with adding another column named Category to set a rank based on college enrollment. high collage enrollment 
score has 0 rank and second rank is assigned 2 and so on.

In [72]:
schools['groups']=pd.cut(schools.COLLEGE_ENROLLMENT,[0,436,870,1305,1740,2175,2610,3045,3915,5000])# we use cut to make groups
#schools['Categories']=schools.groups!=schools.groups.shift()

At this below funtion we are going to set category for every school based on their college admissions. the top schools has a lowe number and the worse has higher number to assign.

In [71]:
def set_category(df):
    if (df['COLLEGE_ENROLLMENT']<=436) and (df['COLLEGE_ENROLLMENT']>0):
        return 8
    elif (df['COLLEGE_ENROLLMENT']<=870) and (df['COLLEGE_ENROLLMENT']>436):
        return 7
    elif(df['COLLEGE_ENROLLMENT']<=1306) and (df['COLLEGE_ENROLLMENT']>870):
        return 6
    elif (df['COLLEGE_ENROLLMENT']<=1740) and (df['COLLEGE_ENROLLMENT']>1306):
        return 5
    elif(df['COLLEGE_ENROLLMENT']<=2175) and (df['COLLEGE_ENROLLMENT']>1740):
        return 4
    elif(df['COLLEGE_ENROLLMENT']<=2610) and (df['COLLEGE_ENROLLMENT']>2175):
        return 3
    elif(df['COLLEGE_ENROLLMENT']<=3045) and (df['COLLEGE_ENROLLMENT']>2610):
        return 2
    elif(df['COLLEGE_ENROLLMENT']<=3915) and (df['COLLEGE_ENROLLMENT']>3045):
        return 1
    elif(df['COLLEGE_ENROLLMENT']>3915):
        return 0                

In [73]:
schools['Categories'] = schools.apply(set_category, axis = 1)

In [79]:
schools.sort_values('Categories', ascending=True).reset_index(drop=True)

,School_ID,NAME_OF_SCHOOL,"Elementary,_Middle,_or_High_School",ZIP_Code,CPS_Performance_Policy_Level,SAFETY_SCORE,Instruction_Score,Leaders_Score_,Teachers_Score,Rate_of_Misconducts_(per_100_students)_,COLLEGE_ENROLLMENT,Latitude,Longitude,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,groups,Categories,Top_schools
0,609720,Albert G Lane Technical High School,HS,60618,Level 1,88.0,52.0,48,53,2.1,4368,41.946617,-87.691056,5,NORTH CENTER,"(3915, 5000]",0,True
1,609756,Marie Sklodowska Curie Metropolitan High School,HS,60632,Level 3,43.0,45.0,33,39,9.1,3320,41.803046,-87.722007,57,ARCHER HEIGHTS,"(3045, 3915]",1,True
2,609734,William Howard Taft High School,HS,60631,Level 2,61.0,32.0,48,53,9.8,2922,41.982989,-87.791538,10,NORWOOD PARK,"(2610, 3045]",2,True
3,609715,Thomas Kelly High School,HS,60632,Level 2,36.0,35.0,61,57,5.8,2883,41.818711,-87.694675,58,BRIGHTON PARK,"(2610, 3045]",2,True
4,609729,Carl Schurz High School,HS,60641,Level 3,48.0,29.0,30,31,13.8,2366,41.946408,-87.735625,16,IRVING PARK,"(2175, 2610]",3,True
5,609738,Lincoln Park High School,HS,60614,Level 2,65.0,49.0,45,45,9.2,2342,41.918304,-87.645974,7,LINCOLN PARK,"(2175, 2610]",3,True
6,610157,Sidney Sawyer Elementary School,ES,60632,Level 2,57.0,39.0,24,29,7.4,1846,41.797522,-87.705043,63,GAGE PARK,"(1740, 2175]",4,False
7,609732,Charles P Steinmetz Academic Centre High School,HS,60634,Level 3,26.0,34.0,36,32,30.4,1890,41.935632,-87.783552,19,BELMONT CRAGIN,"(1740, 2175]",4,False
8,609898,John F Eberhart Elementary School,ES,60629,Level 3,42.0,34.0,42,32,13.1,1777,41.774447,-87.708002,66,CHICAGO LAWN,"(1740, 2175]",4,False
9,609755,Whitney M Young Magnet High School,HS,60607,Level 1,95.0,67.0,52,46,1.2,2166,41.878590,-87.664196,28,NEAR WEST SIDE,"(1740, 2175]",4,False


In [86]:
schools['Top_schools']=schools['Categories'].apply(lambda x: True if x<=4 else False)###assign True and False according to their rank

In [87]:
 for name, community_name, cat, top in zip(schools.NAME_OF_SCHOOL, schools.COMMUNITY_AREA_NAME, schools.Categories, schools.Top_schools):
        if top==True:
                print('School with name {} at community {} with category {} is in best categoris school.'.format(name,
                                                                                                       community_name,
                                                                                                       cat)) 

School with name Albert G Lane Technical High School at community NORTH CENTER with category 0 is in best categoris school.
School with name Carl Schurz High School at community IRVING PARK with category 3 is in best categoris school.
School with name Charles P Steinmetz Academic Centre High School at community BELMONT CRAGIN with category 4 is in best categoris school.
School with name James Shields Elementary School at community BRIGHTON PARK with category 4 is in best categoris school.
School with name John F Eberhart Elementary School at community CHICAGO LAWN with category 4 is in best categoris school.
School with name Kenwood Academy High School at community KENWOOD with category 4 is in best categoris school.
School with name Lincoln Park High School at community LINCOLN PARK with category 3 is in best categoris school.
School with name Marie Sklodowska Curie Metropolitan High School at community ARCHER HEIGHTS with category 1 is in best categoris school.
School with name Sidne

###### As you see we found 12  best schools in Chicago city as printed above

### let's visualize schools based on their category

In [88]:
k=list(schools.Categories.astype(int).unique())
np.arange(len(k))

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [89]:

    k =list(schools.Categories.astype(int).unique())
    x=np.arange(len(k))
    ys = [i + x + (i*x)**2 for i in range(len(k))]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    print(colors_array)

[[5.00000000e-01 0.00000000e+00 1.00000000e+00 1.00000000e+00]
 [2.49019608e-01 3.84105749e-01 9.80634770e-01 1.00000000e+00]
 [1.96078431e-03 7.09281308e-01 9.23289106e-01 1.00000000e+00]
 [2.52941176e-01 9.25637660e-01 8.30184031e-01 1.00000000e+00]
 [5.03921569e-01 9.99981027e-01 7.04925547e-01 1.00000000e+00]
 [7.54901961e-01 9.20905518e-01 5.52364973e-01 1.00000000e+00]
 [1.00000000e+00 7.00543038e-01 3.78411050e-01 1.00000000e+00]
 [1.00000000e+00 3.72701992e-01 1.89801093e-01 1.00000000e+00]
 [1.00000000e+00 1.22464680e-16 6.12323400e-17 1.00000000e+00]]


In [90]:
    # These codes show schools base on their latitude and longitude the high category school would be showm with large radius
    # and gradually getting smaller to the lower category school.

    Chicago_map_cat = folium.Map(location=[latitude, longitude], zoom_start=11)

    k =list(schools.Categories.astype(int).unique())
    x=np.arange(len(k))
    ys = [i + x + (i*x)**2 for i in range(len(k))]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow =[colors.rgb2hex(i) for i in colors_array]

    markers_colors = []
    for lat, lon, poi, cat in zip(schools['Latitude'], schools['Longitude'], schools['COMMUNITY_AREA_NAME'], schools['Categories']):
        label = folium.Popup(str(poi) + ' Category ' + str(cat), parse_html=True)
        if int(cat)==0:
            folium.CircleMarker(
            [lat, lon],
            radius=20,
            popup=label,
            color='darkred',
            fill=True,
            fill_color='darkred',
            fill_opacity=0.7).add_to(Chicago_map_cat)
        elif int(cat)==1:
            folium.CircleMarker(
            [lat, lon],
            radius=15,
            popup=label,
            color='purple',
            fill=True,
            fill_color='purple',
            fill_opacity=0.7).add_to(Chicago_map_cat)
        elif int(cat)==2:
            folium.CircleMarker(
            [lat, lon],
            radius=11,
            popup=label,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.7).add_to(Chicago_map_cat)
        elif int(cat)==3:
            folium.CircleMarker(
            [lat, lon],
            radius=8,
            popup=label,
            color='green',
            fill=True,
            fill_color='green',
            fill_opacity=0.7).add_to(Chicago_map_cat)
        elif int(cat)==4:
            folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.7).add_to(Chicago_map_cat)
        else: 
            folium.CircleMarker(
            [lat, lon],
            radius=2,
            popup=label,
            color=rainbow[cat-1],
            fill=True,
            fill_color=rainbow[cat-1],
            fill_opacity=0.7).add_to(Chicago_map_cat)
    Chicago_map_cat


## 3. Clustering schools 

#### Next, let's group rows by community and by taking the mean of the frequency of occurrence of each category

In [101]:
schools['Instruction_Score'].fillna((schools['Instruction_Score'].mean()), inplace=True)

In [102]:
schools['SAFETY_SCORE'].fillna((schools['SAFETY_SCORE'].mean()), inplace=True)

In [103]:
schools_grouped = schools.groupby('COMMUNITY_AREA_NAME').mean().reset_index()

In [94]:
schools_grouped

,COMMUNITY_AREA_NAME,School_ID,ZIP_Code,SAFETY_SCORE,Instruction_Score,Rate_of_Misconducts_(per_100_students)_,COLLEGE_ENROLLMENT,Latitude,Longitude,COMMUNITY_AREA_NUMBER,Categories,Top_schools
0,ALBANY PARK,610046.750000,60625.625000,59.563109,52.161062,11.962500,858.000000,41.968518,-87.717327,14.0,6.625000,0.000000
1,ARCHER HEIGHTS,609829.500000,60632.000000,45.500000,43.000000,9.700000,2411.500000,41.804285,-87.723913,57.0,3.000000,0.500000
2,ARMOUR SQUARE,610229.666667,60613.666667,43.333333,46.000000,5.733333,486.000000,41.840676,-87.633966,34.0,7.333333,0.000000
3,ASHBURN,610019.500000,60652.000000,45.000000,36.125000,24.600000,810.375000,41.745201,-87.715027,70.0,6.500000,0.000000
4,AUBURN GRESHAM,610004.000000,60620.000000,34.050487,44.228850,30.530000,417.500000,41.743401,-87.653819,71.0,7.600000,0.000000
5,AUSTIN,600994.391304,60648.043478,40.609119,53.720739,25.165217,475.347826,41.889016,-87.762496,25.0,7.434783,0.000000
6,AVALON PARK,609764.333333,60617.666667,28.000000,35.333333,38.700000,507.333333,41.741565,-87.581876,45.0,7.666667,0.000000
7,AVONDALE,610262.250000,60618.000000,56.250000,48.750000,14.850000,910.000000,41.938789,-87.713729,21.0,6.500000,0.000000
8,BELMONT CRAGIN,610041.916667,60637.916667,48.126218,40.155458,8.383333,1198.833333,41.926565,-87.765742,19.0,5.666667,0.083333
9,BEVERLY,610051.500000,60637.250000,70.500000,52.750000,11.900000,409.000000,41.715650,-87.669420,72.0,7.750000,0.000000


In [95]:
schools_grouped_clustering = schools_grouped.drop(['School_ID', 'ZIP_Code'],axis=1)#### remove some columns to cluster data properly

In [96]:
schools_grouped_clustering.head()

,COMMUNITY_AREA_NAME,SAFETY_SCORE,Instruction_Score,Rate_of_Misconducts_(per_100_students)_,COLLEGE_ENROLLMENT,Latitude,Longitude,COMMUNITY_AREA_NUMBER,Categories,Top_schools
0,ALBANY PARK,59.563109,52.161062,11.962500,858.000,41.968518,-87.717327,14.0,6.625000,0.0
1,ARCHER HEIGHTS,45.500000,43.000000,9.700000,2411.500,41.804285,-87.723913,57.0,3.000000,0.5
2,ARMOUR SQUARE,43.333333,46.000000,5.733333,486.000,41.840676,-87.633966,34.0,7.333333,0.0
3,ASHBURN,45.000000,36.125000,24.600000,810.375,41.745201,-87.715027,70.0,6.500000,0.0
4,AUBURN GRESHAM,34.050487,44.228850,30.530000,417.500,41.743401,-87.653819,71.0,7.600000,0.0


In [97]:
######## normalize and regulate scale of each data of dataset#########
from sklearn.preprocessing import StandardScaler

X = schools_grouped_clustering.values[:,1:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

array([[ 5.07936406e-01,  5.00746443e-01, -5.29903731e-01,
         5.88764043e-01,  1.31033782e+00, -6.33110909e-01,
        -1.12480318e+00, -4.73734329e-01, -3.24653911e-01],
       [-4.02476980e-01, -5.17865849e-01, -7.08639913e-01,
         5.35884872e+00, -3.52180782e-01, -7.31275657e-01,
         8.09858287e-01, -5.13271013e+00,  6.60634454e+00],
       [-5.42742003e-01, -1.84297890e-01, -1.02200427e+00,
        -5.53477026e-01,  1.62073172e-02,  6.09449866e-01,
        -2.24960635e-01,  4.36640253e-01, -3.24653911e-01],
       [-4.34845831e-01, -1.28229242e+00,  4.68451409e-01,
         4.42529552e-01, -9.50282914e-01, -5.98832092e-01,
         1.39475594e+00, -6.34388667e-01, -3.24653911e-01],
       [-1.14369213e+00, -3.81230864e-01,  9.36917955e-01,
        -7.63809051e-01, -9.68499881e-01,  3.13529272e-01,
         1.43974807e+00,  7.79369507e-01, -3.24653911e-01],
       [-7.19101355e-01,  6.74165831e-01,  5.13103254e-01,
        -5.86184957e-01,  5.05548612e-01, -1.306383

In [98]:
#########applying clustering to dataset#########
num_clusters = 5

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[4 3 1 0 2 1 2 4 4 1 0 0 2 2 2 0 1 2 4 1 0 0 4 2 4 2 0 0 2 2 2 4 1 2 4 4 2
 4 4 4 1 4 1 1 4 2 1 1 1 1 2 4 1 4 4 2 0 4 2 2 4 2 2 2 1 2 4 2 2 0 2 1 0 2
 4 1 2]


In [105]:
schools_grouped["Labels"] = labels
schools_grouped.head(5)

,COMMUNITY_AREA_NAME,School_ID,ZIP_Code,SAFETY_SCORE,Instruction_Score,Rate_of_Misconducts_(per_100_students)_,COLLEGE_ENROLLMENT,Latitude,Longitude,COMMUNITY_AREA_NUMBER,Categories,Top_schools,Labels
0,ALBANY PARK,610046.750000,60625.625000,59.563109,52.161062,11.962500,858.000,41.968518,-87.717327,14.0,6.625000,0.0,4
1,ARCHER HEIGHTS,609829.500000,60632.000000,45.500000,43.000000,9.700000,2411.500,41.804285,-87.723913,57.0,3.000000,0.5,3
2,ARMOUR SQUARE,610229.666667,60613.666667,43.333333,46.000000,5.733333,486.000,41.840676,-87.633966,34.0,7.333333,0.0,1
3,ASHBURN,610019.500000,60652.000000,45.000000,36.125000,24.600000,810.375,41.745201,-87.715027,70.0,6.500000,0.0,0
4,AUBURN GRESHAM,610004.000000,60620.000000,34.050487,44.228850,30.530000,417.500,41.743401,-87.653819,71.0,7.600000,0.0,2


#### Now it's time to visualize the clustered communities according to their performance of scools in Chicago city

In [107]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(schools_grouped['Latitude'], schools_grouped['Longitude'], schools_grouped['COMMUNITY_AREA_NAME'], schools_grouped['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
#colormap = branca.colormap.linear.YlOrRd_09.scale(0, 8500)
#colormap = colormap.to_step(index=[0, 1000, 3000, 5000, 8500])
#colormap.caption = 'Clustering of Schools in Chicago (year ending June 2018)'
#colormap.add_to(map_clusters)      
map_clusters

### 4. Conclusion

If you noticed and cluster with label 3 and cluster with label 4 are better schools and the label 3 is the best schools in chicago city, so if parents have intent to buy home, it's better to invest both on money and future of their great children.
from my point of view ARCHER HEIGHTS community is the best place to buy home and after this cluster 4 communit's with orange color and cluster 0 with red colors would be contain good schools in Chicago city.These are especially relevant info from clustering technique, and provide us to smartly make a proper decision.

### hope you like and enjoy this analysing. Good Luck